In [ ]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='sitedropdown',
                                value='ALL',
                options=[
                    {'label': 'ALL', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                ],
                placeholder="Select a Launch Site here",
                searchable=True
                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={0: '0',
                       10000: '10000'},
                value=[0, 9600]), 
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='sitedropdown', component_property='value'))

def get_pie_chart(sitedropdown):
        filtered_df = spacex_df
        if sitedropdown == 'ALL':
                fig = px.pie(spacex_df, values='class', 
                names='Launch Site', 
                title='All')
                return fig
        
        elif sitedropdown == 'CCAFS LC-40':
                spacex_df1 = spacex_df.loc[spacex_df['Launch Site']=='CCAFS LC-40']
                fig  = px.pie(spacex_df1, values=spacex_df1['class'].value_counts().values, 
                names=spacex_df1['class'].value_counts().index, 
                title='CCAFS LC-40')
                return fig
        elif sitedropdown == 'VAFB SLC-4E':
                spacex_df2 = spacex_df.loc[spacex_df['Launch Site']=='VAFB SLC-4E']
                fig = px.pie(spacex_df2, values==spacex_df2['class'].value_counts().values, 
                names=spacex_df2['class'].value_counts().index, 
                title='VAFB SLC-4E')
                return fig
        elif sitedropdown == 'KSC LC-39A':
                spacex_df3 = spacex_df.loc[spacex_df['Launch Site']=='KSC LC-39A']
                fig = px.pie(spacex_df3, values=spacex_df3['class'].value_counts().values, 
                names=spacex_df3['class'].value_counts().index, 
                title='KSC LC-39A')
                return fig
        else:
                spacex_df4=spacex_df.loc[spacex_df['Launch Site']=='CCAFS SLC-40']
                fig = px.pie(spacex_df4, values=spacex_df4['class'].value_counts().values, 
                names=spacex_df4['class'].value_counts().index, 
                title='CCAFS SLC-40')
                return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='sitedropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(sitedropdown,payloadslider):
        filtered_df = spacex_df
        if sitedropdown == 'ALL':
            spacex_df1 = spacex_df.loc[(spacex_df['Payload Mass (kg)']>=payloadslider[0])&(spacex_df['Payload Mass (kg)']<=payloadslider[1])]
            fig1 = px.scatter(spacex_df1, x="Payload Mass (kg)", y="class", color="Booster Version Category")
            return fig1
        
        else: 
            spacex_df1 = spacex_df.loc[(spacex_df['Payload Mass (kg)']>=payloadslider[0])&(spacex_df['Payload Mass (kg)']<=payloadslider[1])&(spacex_df['Launch Site']==sitedropdown)]      
            fig1 = px.scatter(spacex_df1, x="Payload Mass (kg)", y="class", color="Booster Version Category",title=sitedropdown)
            return fig1
# Run the app
if __name__ == '__main__':
    app.run_server()
